# Alignment and variant detection

## Alignment

Download the data and alignment software.

In [ ]:
wget https://github.com/cb2edu/CB2-101-BioComp/raw/2020/08-Alignment_and_variant_calling/data/sample_data.tar.gz
tar -xvzf sample_data.tar.gz

Download BWA.

In [ ]:
wget -O bwa.tar.bz2 "https://downloads.sourceforge.net/project/bio-bwa/bwa-0.7.17.tar.bz2?r=https%3A%2F%2Fsourceforge.net%2Fprojects%2Fbio-bwa%2Ffiles%2Flatest%2Fdownload&ts=1605561416"
tar -xvjf bwa.tar.bz2

Compile BWA.

In [ ]:
cd bwa-0.7.17
make
cd ..

Index the reference genome using bwa.

In [ ]:
./bwa-0.7.17/bwa
./bwa-0.7.17/bwa index sample_data/chr20.fa -p chr20

Carry out the alignment.

In [ ]:
./bwa-0.7.17/bwa mem chr20 sample_data/SRR765989_F1.fastq sample_data/SRR765989_F2.fastq | samtools view -bS >SRR765989.bam

Sort the bam file by coordinates.

In [ ]:
samtools sort SRR765989.bam >SRR765989.sorted.bam

### Generating pileup

In [ ]:
samtools mpileup -f sample_data/chr20.fa SRR765989.sorted.bam | head -n 50

Each line corresponds to a position on the genome. 

The columns are: chromosome, position, reference base, read depth, read bases (dot . and comma , indicate match on the forward and on the reverse strand; ACGTN and acgtn a mismatch on the forward and the reverse strand) and the final column is the base qualities encoded into characters. The caret symbol ^ marks the start of a read, the dollar sign $ the end of a read, deleted bases are represented by asterisk *.

This output can be used for a simple consensus calling. One rarely needs this type of output. Instead, for a more sophisticated variant calling method, see the next section.

### Exercises
Modify the `mpileup` command above and answer the following questions.

**Q1:** What is the read depth at position 60158? (Rather than scrolling to the position, use grep to find the position. Before doing this, check if the above command has finished running, if not, you may need to interrupt it)

**Q2:** What is the reference base and the alternate base at the 60158? 

**Q3:** How many reference and how many non-reference bases are there?

## Variant detection

Create an index for the reference.

In [ ]:
samtools faidx sample_data/chr20.fa

Download GATK.

In [ ]:
wget https://github.com/broadinstitute/gatk/releases/download/4.1.9.0/gatk-4.1.9.0.zip

In [ ]:
unzip gatk-4.1.9.0.zip

Download Picard.

In [ ]:
#cd ..
wget https://github.com/broadinstitute/picard/releases/download/2.23.8/picard.jar

Marduplicates using picard.

In [ ]:
java -jar picard.jar MarkDuplicates INPUT=SRR765989.sorted.bam OUTPUT=SRR765989.dup.bam \
METRICS_FILE=picard_metrics.txt VALIDATION_STRINGENCY=LENIENT

Create a sequency dictionary, a required step for running GATK.

In [ ]:
cd sample_data/
java -jar ../picard.jar CreateSequenceDictionary REFERENCE=chr20.fa OUTPUT=chr20.dict

Add readgroups to bam file. For the description of readgroups look here: https://software.broadinstitute.org/gatk/documentation/article.php?id=6472

In [ ]:
cd ..

In [ ]:
java -jar picard.jar AddOrReplaceReadGroups I=SRR765989.dup.bam O=SRR765989.dup.rg.bam RGID=4 RGLB=lib1 RGPL=illumina RGPU=unit1 RGSM=20 CREATE_INDEX=true

We are now ready to run GATK. First well realign the bam files (No longer needed for GATK v4). Use the following code if you are using an old version of GATK.

```
## create realignment target
java -jar ../GenomeAnalysisTK.jar -T RealignerTargetCreator -R gatk_ref/chr20.fa -o SRR765989.paired.bam.list -I SRR765989.dup.rg.bam

# Realign
java -jar ../GenomeAnalysisTK.jar -I SRR765989.dup.rg.bam -R gatk_ref/chr20.fa -T IndelRealigner -targetIntervals SRR765989.paired.bam.list -o SRR765989.realigned.bam
```

We have to use a base recalibration stage before we can call SNV. Check the `run.sh` file for this step. For this excersize we will skip that stage.

#### Download dbSNP
This is prepackaged for this class for easy access. The original data can be accessed from GATK resource bundle.

In [ ]:
wget http://cmb.path.uab.edu/training/2020/files/Homo_sapiens_assembly38.dbsnp138.vcf.tar.xz

In [ ]:
tar -xvJf Homo_sapiens_assembly38.dbsnp138.vcf.tar.xz

#### Recalibrate the bases

In [ ]:
gatk-4.1.9.0/gatk BaseRecalibrator  -R sample_data/chr20.fa  -I SRR765989.dup.rg.bam --known-sites Homo_sapiens_assembly38.dbsnp138.vcf -O SRR765989.recal_data.csv

In [ ]:
gatk-4.1.9.0/gatk ApplyBQSR -R sample_data/chr20.fa  -I SRR765989.dup.rg.bam --bqsr-recal-file SRR765989.recal_data.csv -O SRR765989.recal.bam

Now we can call SNPs.

In [ ]:
gatk-4.1.9.0/gatk HaplotypeCaller -R sample_data/chr20.fa  -I SRR765989.recal.bam -O raw_variants.vcf --dbsnp Homo_sapiens_assembly38.dbsnp138.vcf

For full happlotype caller check the `run.sh` file.

#### VQSR

We cannot run VQSR is our dataset. The coverage is too low. The command is for demonstration only. For details see: https://gatk.broadinstitute.org/hc/en-us/articles/360035531612-Variant-Quality-Score-Recalibration-VQSR-

```
gatk-4.1.9.0/gatk VariantRecalibrator -R sample_data/chr20.fa -V raw_variants.vcf \
--resource:hapmap,known=false,training=true,truth=true,prior=15.0 hapmap_3.3.hg38.vcf.gz \
--resource:dbsnp,known=true,training=false,truth=false,prior=2.0 Homo_sapiens_assembly38.dbsnp138.vcf \
-mode SNP \
--tranches-file output.tranches \
-O output.vqsr \
-an AF

gatk ApplyVQSR \
   -R Homo_sapiens_assembly38.fasta \
   -V input.vcf.gz \
   -O output.vcf.gz \
   --truth-sensitivity-filter-level 99.0 \
   --tranches-file output.tranches \
   --recal-file output.recal \
   -mode SNP
```

In [ ]:
cat raw_variants.vcf

For real-life scenario, you need run a VQSR filtering step. See `run.sh` for details.

### Exercises

Look at the content of the VCF file produced above and answers the questions that follow (Remember to wait until the command above has finished running!). 

In [ ]:
cat raw_variants.vcf | grep 64048910

**Q1:** What is the reference base and the alternate base at position 64048910?

**Q2:** What is the total raw read depth at position 64048910? 

**Q3:** What is the number of high-quality reads supporting the SNP call at position 64048910? How many reads support the reference allele and how many support the alternate allele? 

**Hint:** Look up the AD tag in the FORMAT column: the first value gives the number of reference reads and the second gives the number of non-reference reads.

**Q4:** What sort of event is happening at position 64048910? 

## Variant annotation

In [ ]:
wget https://snpeff.blob.core.windows.net/versions/snpEff_latest_core.zip

In [ ]:
unzip snpEff_latest_core.zip

Find which database to use.

In [ ]:
java -Xmx2g -Djava.io.tmpdir=. -jar snpEff/snpEff.jar databases | grep -i Homo_sapiens

We will use `hg19`.

In [ ]:
java -Xmx2g -Djava.io.tmpdir=. -jar snpEff/snpEff.jar -v hg19 raw_variants.vcf >SRR765989.snpeff.vcf